# Octopy Features Generation

Vamos a crear las features que me permite obtener Octopy estas son las siguientes:

* Coefs para equipo local y visitante
* Probs para equipo local y visitante

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
import pandas as pd
import time
from datetime import *
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from logistic_model import LogisticRegression, LogisticModel
from implied import ImpliedProbability
from sklearn.metrics import plot_confusion_matrix, accuracy_score

In [2]:
def connect_database(db):
    con = sqlite3.connect(db)
    cursor = con.cursor()
    return cursor, con


def leer_partidos(con):
    partidos = pd.read_sql_query('SELECT * FROM partidos', con) #, index_col='fecha')
    return partidos

cur, con = connect_database('../../Clasificacion.db')

partidos = leer_partidos(con)

In [3]:
partidos['fecha'] = partidos['timestamp']
partidos['fecha'] = partidos['fecha'].map(lambda X: int(datetime.fromtimestamp(int(X)).date().strftime('%Y%m%d')))
partidos = partidos.sort_values(by='fecha')

In [4]:
temporadas = partidos['temporada'].unique()
dias = partidos['fecha'].unique()
divisiones = partidos['division'].unique()

In [5]:
for div in divisiones:
    partidos_div = partidos[partidos['division'] == div]

    for temp in temporadas:
        partidos_temp = partidos_div[partidos_div['temporada'] == temp]
        equipos = partidos_temp['equipo_local'].unique()
        dias = partidos_temp[partidos_temp['jornada'] > 2]['fecha'].unique()

        for d in dias:
            partidos_dia = partidos_temp[partidos_temp['fecha'] <= d]
            model = LogisticModel()
            model.fit(partidos_dia['equipo_local'],partidos_dia['equipo_visitante'],partidos_dia['goles_local'],partidos_dia['goles_visitante'])
            coefs = model.get_coef()
            partidos_hoy = partidos_dia[partidos_dia['fecha'] == d]
            for index, row in partidos_hoy.iterrows():
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_home_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_draw'] = \
                    coefs.loc['home_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'local_away_wins'] = \
                    coefs.loc['home_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_home_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['home wins'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_draw'] = \
                    coefs.loc['away_'+row.equipo_local]['draw'][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'visitante_away_wins'] = \
                    coefs.loc['away_'+row.equipo_local]['away wins'][0]

                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_local'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_local][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_equipo_visitante'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)[row.equipo_visitante][0]
                partidos.at[partidos['id_partido'] == row.id_partido, 'probs_draw'] = \
                    model.predict_proba(row.equipo_local, row.equipo_visitante)['draw'][0]

        print(temp + ' ' + str(div))


1970-71 2
1971-72 2
1972-73 2
1973-74 2
1974-75 2
1975-76 2
1976-77 2
1977-78 2
1978-79 2
1979-80 2
1980-81 2
1981-82 2
1982-83 2
1983-84 2
1984-85 2
1985-86 2
1986-87 2
1987-88 2
1988-89 2
1989-90 2
1990-91 2
1991-92 2
1992-93 2
1993-94 2
1994-95 2
1995-96 2
1996-97 2
1997-98 2
1998-99 2
1999-00 2
2000-01 2
2001-02 2
2002-03 2
2003-04 2
2004-05 2
2005-06 2
2006-07 2
2007-08 2
2008-09 2
2009-10 2
2010-11 2
2011-12 2
2012-13 2
2013-14 2
2014-15 2
2015-16 2
2016-17 2
2017-18 2
2018-19 2
2019-20 2
2020-21 2
2021-22 2
1970-71 1
1971-72 1
1972-73 1
1973-74 1
1974-75 1
1975-76 1
1976-77 1
1977-78 1
1978-79 1
1979-80 1
1980-81 1
1981-82 1
1982-83 1
1983-84 1
1984-85 1
1985-86 1
1986-87 1
1987-88 1
1988-89 1
1989-90 1
1990-91 1
1991-92 1
1992-93 1
1993-94 1
1994-95 1
1995-96 1
1996-97 1
1997-98 1
1998-99 1
1999-00 1
2000-01 1
2001-02 1
2002-03 1
2003-04 1
2004-05 1
2005-06 1
2006-07 1
2007-08 1
2008-09 1
2009-10 1
2010-11 1
2011-12 1
2012-13 1
2013-14 1
2014-15 1
2015-16 1
2016-17 1
2017-18 1


In [6]:
partidos.to_csv('partidos_with_octopy_probabilites.csv')

## Convert String Date to Timestamp

In [7]:
import time
import datetime

In [8]:
time.mktime(datetime.datetime.strptime("23/5/2021", "%d/%m/%Y").timetuple())

1621720800.0